In [1]:
from qiskit import *
from qiskit.visualization import *
from qiskit.quantum_info import *
import random 
%pip install qiskit-aer
import matplotlib
import socket
import time

def measure_and_count(circuit, N):
    for i in range(N):
        circuit.measure(i, i)
    sim = Aer.get_backend('aer_simulator')
    job = execute(circuit, sim)
    result = job.result()
    counts = result.get_counts()
    return counts


In [2]:
def publish_alice_bob_keys(n_node):
    KeyRandom = random.choices([0,1], k=16)
    KeyToBeTransimtted = "".join([str(a) for a in KeyRandom])

    # print (KeyToBeTransimtted)

    N = len(KeyToBeTransimtted)
    BobFullKeyReceived = False 
    BobsKey = [-1]*N
    iterations = 0 
    state = None

    for i in range(2*N):
        QuantumBB84Circuit = QuantumCircuit(N,N)
        for idx, i in enumerate(KeyToBeTransimtted):
            if i == "1":
                QuantumBB84Circuit.x(idx)

        QuantumBB84Circuit.barrier()

        # Alice choses a basis randomly.
        alice_basis_choice = random.choices([0,1], k=N)
        for idx, c in enumerate(alice_basis_choice):
            if c == 1:
                QuantumBB84Circuit.h(idx)

        QuantumBB84Circuit.barrier()

        # Bob choses a basis randomly.
        bob_basis_choice = random.choices([0,1], k=N)
        for idx, c in enumerate(bob_basis_choice):
            if c == 1:
                QuantumBB84Circuit.h(idx)
                
        QuantumBB84Circuit.barrier()

        z = measure_and_count(QuantumBB84Circuit, N)
        
        ## publicly share alice and bob basis.
        reference = list(z.keys())
        reference = [k[::-1] for k in reference]

        def get_all_values_at_index(reference, bit_no): 
            ans = reference[0][bit_no]

            for i in range(1, len(reference)):
                if ans != reference[i][bit_no]:
                    return -1
            
            return ans 

        for idx, (alice_choice, bob_choice) in enumerate(zip(alice_basis_choice, bob_basis_choice)):
            if alice_choice == bob_choice:
                ans = get_all_values_at_index(reference, idx)
                if BobsKey[idx] == -1 and ans != -1:
                    BobsKey[idx] = ans

        iterations += 1

        state = QuantumBB84Circuit

        if -1 not in BobsKey:
            break

    print("Alice key =>", KeyToBeTransimtted, "\nBobs  key =>", "".join(BobsKey), '\nretrieved in' , iterations, 'iterations')

    # %pip install matplotlib
    # %pip install pylatexenc
    # state.draw(output="mpl", scale=0.5, fold=1000)


    # n_node = int(input("enter the number of nodes"))
    bob_key="".join(BobsKey)
    alice_key=KeyToBeTransimtted



    for i in range(n_node, 0, -1):
    
        if i%2 == 0:
            client_socket = socket.socket()
            port_num = "700" + str(i)
            print(port_num)
            client_socket.connect(("127.0.0.1", int(port_num)))

            message = alice_key
            client_socket.sendall(message.encode())
            print("Bob sent on port ", port_num)
            client_socket.close()
            
        else:
            client_socket = socket.socket()
            port_num = "700" + str(i)
            print(port_num)
            client_socket.connect(("127.0.0.1", int(port_num)))

            message = bob_key
            client_socket.sendall(message.encode())
            print("Alice sent on port ", port_num)
            client_socket.close()


In [4]:
n = int(input("enter number of nodes"))

while True:
    publish_alice_bob_keys(n)
    print("\n")
    time.sleep(1)
    

Alice key => 1100100010101110 
Bobs  key => 1100100010101110 
retrieved in 5 iterations
7004
Bob sent on port  7004
7003
Alice sent on port  7003
7002
Bob sent on port  7002
7001
Alice sent on port  7001


Alice key => 1010111010111110 
Bobs  key => 1010111010111110 
retrieved in 4 iterations
7004
Bob sent on port  7004
7003
Alice sent on port  7003
7002
Bob sent on port  7002
7001
Alice sent on port  7001


Alice key => 0010110011111010 
Bobs  key => 0010110011111010 
retrieved in 6 iterations
7004
Bob sent on port  7004
7003
Alice sent on port  7003
7002
Bob sent on port  7002
7001
Alice sent on port  7001


Alice key => 1010001000111101 
Bobs  key => 1010001000111101 
retrieved in 5 iterations
7004
Bob sent on port  7004
7003
Alice sent on port  7003
7002
Bob sent on port  7002
7001
Alice sent on port  7001


Alice key => 0011010010010011 
Bobs  key => 0011010010010011 
retrieved in 6 iterations
7004
Bob sent on port  7004
7003
Alice sent on port  7003
7002
Bob sent on port  7002
70

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it